### Adjacent Countries by Maritime Boundaries Ontology
https://en.wikipedia.org/wiki/List_of_countries_and_territories_by_maritime_boundaries

In [57]:
import rdflib

graph = rdflib.Graph()
adjacent_with = rdflib.URIRef('http://cgraf.pucp.edu.pe/adjacent_with')
country_name = rdflib.URIRef('http://cgraf.pucp.edu.pe/country_name')
located_in = rdflib.URIRef('http://cgraf.pucp.edu.pe/located_in')

peru = rdflib.URIRef('http://cgraf.pucp.edu.pe/peru')
peruCountry = rdflib.Literal('PERU')

chile = rdflib.URIRef('http://cgraf.pucp.edu.pe/chile')
chileCountry = rdflib.Literal("CHILE")

ecuador = rdflib.URIRef('http://cgraf.pucp.edu.pe/ecuador')
ecuadorCountry = rdflib.Literal("ECUADOR")

sudamerican_plate = rdflib.URIRef('http://cgraf.pucp.edu.pe/sudamerican_plate')

graph.add((peru, country_name, peruCountry))
graph.add((chile, country_name, chileCountry))
graph.add((ecuador, country_name, ecuadorCountry))

graph.add((peru, adjacent_with, chile))
graph.add((peru, adjacent_with, ecuador))
graph.add((peru, located_in, sudamerican_plate))
graph.add((ecuador, located_in, sudamerican_plate))
graph.add((chile, located_in, sudamerican_plate))

# print("All objects")
# query = 'select * { ?s ?p ?o }'
# for row in graph.query(query):
#     print(row)
# print()

print("List of countries in the same plate")
query = 'select ?o {' \
        ' ?s <http://cgraf.pucp.edu.pe/located_in> <http://cgraf.pucp.edu.pe/sudamerican_plate> . ' \
        ' ?s <http://cgraf.pucp.edu.pe/country_name> ?o }'
for row in graph.query(query):
    print(row[0].value)
print()

print("List of countries adjacents to PERU")
query = 'select ?o {' \
        ' <http://cgraf.pucp.edu.pe/peru> <http://cgraf.pucp.edu.pe/adjacent_with> ?s . ' \
        ' ?s <http://cgraf.pucp.edu.pe/country_name> ?o }'
for row in graph.query(query):
    print(row[0].value)

# graph.serialize("graph.rdf")
# g1 = rdflib.Graph()
# g1.parse("graph.rdf", format="xml")
# x1 = g1.queryuery(query)
# print list(x1)

List of countries in the same plate
CHILE
PERU
ECUADOR

List of countries adjacents to PERU
ECUADOR
CHILE


In [17]:
cdf = core.read_country_csv()
cdf[cdf.COUNTRY == 'PERU']

,COUNTRY,ADJACENT
571,PERU,CHILE
572,PERU,ECUADOR


### Volcanos & Tsunamis

In [43]:
import pandas as pd
import numpy as np
import core

tdf = core.read_tsunami_csv()

date = '1678/6/18'
countries = list(['PERU', 'CHILE'])
days = 8

tdf['NOW'] = date
tdf['NOW'] = pd.to_datetime(tdf['NOW'], format='%Y/%m/%d')
tdf['DAYS'] = np.abs(tdf['NOW'] - tdf['DATE'])
tdf['DAYS'] = tdf['DAYS'].dt.days
tdf.query("COUNTRY in @countries and DAYS < @days")

,DATE,COUNTRY,NOW,DAYS
ID,,,,
T328,1678-06-18,PERU,1678-06-18,0.0


### Tsunami after Volcano in a period of time in the Same country

In [6]:
import core
import pandas as pd
import numpy as np

vdf = core.read_volcano_csv()
tdf = core.read_tsunami_csv()
cdf = core.read_country_csv()

results = pd.DataFrame(columns=('ID', 'DATE', 'COUNTRY', 'DAYS', 'GROUP'))

# count = 0
# max_count = 5
days = 8
group = 1
for row in vdf.itertuples():
    date = row.DATE
    countries = list([row.COUNTRY])
    
    tdf['NOW'] = date
    tdf['NOW'] = pd.to_datetime(tdf['NOW'], format='%Y/%m/%d')
    tdf['DAYS'] = np.abs(tdf['NOW'] - tdf['DATE'])
    tdf['DAYS'] = tdf['DAYS'].dt.days
    tdf['GROUP'] = group
    f = tdf.query("COUNTRY in @countries and DAYS < @days")
    if not f.empty:
        results.loc[len(results)] = [row.ID, row.DATE, row.COUNTRY, 0, group]
        results = results.append(pd.DataFrame(f, columns=['ID', 'DATE', 'COUNTRY', 'DAYS', 'GROUP']), ignore_index=True)
        group += 1
 
    # count = count + 1
    # if count > max_count:
    #     break

results.to_csv('volcan-tsunami-samecountry-%sdays.csv' % (days))
results

,ID,DATE,COUNTRY,DAYS,GROUP
0,V134,1690-02-03,ITALY,0,1
1,T347,1690-02-03,ITALY,0,1
2,V137,1693-01-09,ITALY,0,2
3,T353,1693-01-09,ITALY,0,2
4,T354,1693-01-11,ITALY,2,2
5,V138,1693-02-13,ICELAND,0,3
6,T355,1693-02-13,ICELAND,0,3
7,V143,1706-05-05,SPAIN,0,4
8,T374,1706-05-05,SPAIN,0,4
9,V148,1714-06-30,ITALY,0,5


In [8]:
import core
import pandas as pd
import numpy as np

cdf = core.read_country_csv()
l = list(cdf[cdf.COUNTRY == 'PERU'].ADJACENT)
l.append('PERU')
print(l)

['CHILE', 'ECUADOR', 'PERU']


### Tsunami after Volcano in a period of time in the Same & Adjacent countries

In [7]:
import core
import pandas as pd
import numpy as np

vdf = core.read_volcano_csv()
tdf = core.read_tsunami_csv()
cdf = core.read_country_csv()

results = pd.DataFrame(columns=('ID', 'DATE', 'COUNTRY', 'DAYS', 'GROUP'))

days = 8
group = 1
for row in vdf.itertuples():
    date = row.DATE
    countries = list(cdf[cdf.COUNTRY == row.COUNTRY].ADJACENT)
    countries.append(row.COUNTRY)
    
    tdf['NOW'] = date
    tdf['NOW'] = pd.to_datetime(tdf['NOW'], format='%Y/%m/%d')
    tdf['DAYS'] = np.abs(tdf['NOW'] - tdf['DATE'])
    tdf['DAYS'] = tdf['DAYS'].dt.days
    tdf['GROUP'] = group
    f = tdf.query("COUNTRY in @countries and DAYS < @days")
    if not f.empty:
        results.loc[len(results)] = [row.ID, row.DATE, row.COUNTRY, 0, group]
        results = results.append(pd.DataFrame(f, columns=['ID', 'DATE', 'COUNTRY', 'DAYS', 'GROUP']), ignore_index=True)
        group += 1

results.to_csv('volcan-tsunami-neighbors-%sdays.csv' % (days))
results

,ID,DATE,COUNTRY,DAYS,GROUP
0,V134,1690-02-03,ITALY,0,1
1,T347,1690-02-03,ITALY,0,1
2,V137,1693-01-09,ITALY,0,2
3,T353,1693-01-09,ITALY,0,2
4,T354,1693-01-11,ITALY,2,2
5,V138,1693-02-13,ICELAND,0,3
6,T355,1693-02-13,ICELAND,0,3
7,V143,1706-05-05,SPAIN,0,4
8,T374,1706-05-05,SPAIN,0,4
9,V148,1714-06-30,ITALY,0,5


### Tsunami after Volcano in a period of time in Adjacent countries only

In [13]:
import core
import pandas as pd
import numpy as np

vdf = core.read_volcano_csv()
tdf = core.read_tsunami_csv()
cdf = core.read_country_csv()

results = pd.DataFrame(columns=('ID', 'DATE', 'COUNTRY', 'DAYS', 'GROUP'))

days = 8
group = 1
for row in vdf.itertuples():
    date = row.DATE
    countries = list(cdf[cdf.COUNTRY == row.COUNTRY].ADJACENT)
    # countries.append(row.COUNTRY)
    
    tdf['NOW'] = date
    tdf['NOW'] = pd.to_datetime(tdf['NOW'], format='%Y/%m/%d')
    tdf['DAYS'] = np.abs(tdf['NOW'] - tdf['DATE'])
    tdf['DAYS'] = tdf['DAYS'].dt.days
    tdf['GROUP'] = group
    f = tdf.query("COUNTRY in @countries and DAYS < @days")
    if not f.empty:
        results.loc[len(results)] = [row.ID, row.DATE, row.COUNTRY, 0, group]
        results = results.append(pd.DataFrame(f, columns=['ID', 'DATE', 'COUNTRY', 'DAYS', 'GROUP']), ignore_index=True)
        group += 1

results.to_csv('volcan-tsunami-neighbors.only-%sdays.csv' % (days))
results

,ID,DATE,COUNTRY,DAYS,GROUP
0,V201,1794-06-15,ITALY,0,1
1,T576,1794-06-11,GREECE,4,1
2,V342,1901-08-09,VANUATU,0,2
3,T1331,1901-08-09,NEW CALEDONIA,0,2
4,V404,1929-06-17,JAPAN,0,3
5,T1661,1929-06-13,PHILIPPINES,4,3
6,V414,1933-12-24,JAPAN,0,4
7,T1716,1933-12-25,PHILIPPINES,1,4
8,V420,1938-06-05,PHILIPPINES,0,5
9,T1762,1938-06-10,JAPAN,5,5


### Volcano after Volcano in a period of time in the Same & Adjacent countries

In [9]:
import core
import pandas as pd
import numpy as np

vdf = core.read_volcano_csv()
v2df = core.read_volcano_csv()

results = pd.DataFrame(columns=('ID', 'DATE', 'COUNTRY', 'DAYS', 'GROUP'))

days = 8
group = 1
for row in vdf.itertuples():
    date = row.DATE
    countries = list(cdf[cdf.COUNTRY == row.COUNTRY].ADJACENT)
    id = row.ID
    countries.append(row.COUNTRY)
    
    v2df['NOW'] = date
    v2df['NOW'] = pd.to_datetime(v2df['NOW'], format='%Y/%m/%d')
    v2df['DAYS'] = np.abs(v2df['NOW'] - v2df['DATE'])
    v2df['DAYS'] = v2df['DAYS'].dt.days
    v2df['GROUP'] = group
    f = v2df.query("COUNTRY in @countries and DAYS < @days and not ID == @id")
    r = results.query("ID == @id")  # repeated 
    if not f.empty and r.empty:
        results.loc[len(results)] = [row.ID, row.DATE, row.COUNTRY, 0, group]
        results = results.append(pd.DataFrame(f, columns=['ID', 'DATE', 'COUNTRY', 'DAYS', 'GROUP']), ignore_index=True)
        group += 1

results.to_csv('volcan-volcan-neighbors-%sdays.csv' % (days))
results

,ID,DATE,COUNTRY,DAYS,GROUP
0,V229,1826-10-11,INDONESIA,0,1
1,V230,1826-10-11,INDONESIA,0,1
2,V343,1902-05-05,MARTINIQUE,0,2
3,V345,1902-05-08,MARTINIQUE,3,2
4,V414,1933-12-24,JAPAN,0,3
5,V415,1933-12-25,PHILIPPINES,1,3
6,V451,1952-09-16,JAPAN,0,4
7,V452,1952-09-23,JAPAN,7,4
8,V453,1952-09-24,JAPAN,0,5
9,V452,1952-09-23,JAPAN,1,5


### Tsunami after Tsunami in a period of time in the Same & Adjacent countries

In [14]:
import core
import pandas as pd
import numpy as np

vdf = core.read_tsunami_csv()
v2df = core.read_tsunami_csv()

results = pd.DataFrame(columns=('ID', 'DATE', 'COUNTRY', 'DAYS', 'GROUP'))

days = 8
group = 1
for row in vdf.itertuples():
    date = row.DATE
    countries = list(cdf[cdf.COUNTRY == row.COUNTRY].ADJACENT)
    id = row.ID
    countries.append(row.COUNTRY)
    
    v2df['NOW'] = date
    v2df['NOW'] = pd.to_datetime(v2df['NOW'], format='%Y/%m/%d')
    v2df['DAYS'] = np.abs(v2df['NOW'] - v2df['DATE'])
    v2df['DAYS'] = v2df['DAYS'].dt.days
    v2df['GROUP'] = group
    f = v2df.query("COUNTRY in @countries and DAYS < @days and not ID == @id")
    r = results.query("ID == @id")  # repeated 
    if not f.empty and r.empty:
        results.loc[len(results)] = [row.ID, row.DATE, row.COUNTRY, 0, group]
        results = results.append(pd.DataFrame(f, columns=['ID', 'DATE', 'COUNTRY', 'DAYS', 'GROUP']), ignore_index=True)
        group += 1

results.to_csv('tsunami-tsunami-neighbors-%sdays.csv' % (days))
results

,ID,DATE,COUNTRY,DAYS,GROUP
0,T353,1693-01-09,ITALY,0,1
1,T354,1693-01-11,ITALY,2,1
2,T377,1707-10-28,JAPAN,0,2
3,T378,1707-10-28,JAPAN,0,2
4,T458,1755-11-01,PORTUGAL,0,3
5,T459,1755-11-02,PORTUGAL,1,3
6,T484,1763-03-11,JAPAN,0,4
7,T485,1763-03-15,JAPAN,4,4
8,T539,1783-02-05,ITALY,0,5
9,T540,1783-02-05,ITALY,0,5
